<a href="https://colab.research.google.com/github/TrinhHoangKhang/AIO-2024/blob/main/Homework/Module2/Projects/Image%20retrieval%20-%20Vector%20database.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Dowload dataset
!gdown 1msLVo0g0LFmL9-qZ73vq9YEVZwbzOePF
!unzip data

# Dowload packages
!pip install open-clip-torch
!pip install chromadb

Downloading...
From (original): https://drive.google.com/uc?id=1msLVo0g0LFmL9-qZ73vq9YEVZwbzOePF
From (redirected): https://drive.google.com/uc?id=1msLVo0g0LFmL9-qZ73vq9YEVZwbzOePF&confirm=t&uuid=e85f6e43-6df6-4a4c-b539-da47b9d12bea
To: /content/data.zip
100% 76.1M/76.1M [00:00<00:00, 213MB/s]
Archive:  data.zip
replace data/test/American_egret/n02009912_2473.JPEG? [y]es, [n]o, [A]ll, [N]one, [r]ename: Requirement already satisfied: open-clip-torch in /usr/local/lib/python3.10/dist-packages (2.26.1)


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import os
from chromadb.utils.embedding_functions import OpenCLIPEmbeddingFunction
from PIL import Image
from tqdm import tqdm
import chromadb

In [ ]:
embeddingFunction = OpenCLIPEmbeddingFunction()
def create_embedding_one_image(image):
  result = embeddingFunction._encode_image(image)
  return result

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:99: UserWarning: 
Error while fetching `HF_TOKEN` secret value from your vault: 'Requesting secret HF_TOKEN timed out. Secrets can only be fetched when running from the Colab UI.'.
You are not authenticated with the Hugging Face Hub in this notebook.
If the error persists, please let us know by opening an issue on GitHub (https://github.com/huggingface/huggingface_hub/issues/new).
  warnings.warn(


#### Helper function

In [ ]:
POOL_PATH = '/content/data/train'
IMAGE_SIZE = (448, 448)

In [ ]:
# A function to get all the train images's path
def get_all_train_path(pool_path):
  images_path = []
  for class_name in os.listdir(pool_path):
    class_path = os.path.join(pool_path, class_name)
    for image_name in os.listdir(class_path):
      image_path = os.path.join(class_path, image_name)
      images_path.append(image_path)
  return np.array(images_path)

# A function to read image
def read_img(path, size):
    im = Image.open(path).convert('RGB').resize(size)
    return np.array(im)

#### Create the vector database

In [ ]:
def add_embedding(colection, files_path):
  ids = []
  embeddings = []
  for i, path in tqdm(enumerate(files_path)):
    image = read_img(path, IMAGE_SIZE)
    try:
      image_embedded = create_embedding_one_image(image)
    except:
      print("Error in this step")
    ids.append(f'id_{i}')
    embeddings.append(image_embedded)
    print(f'Added {path}')

  colection.add(
      ids=ids,
      embeddings=embeddings,
  )

L2 Norm

In [ ]:
# Create a Chroma client
chroma_client = chromadb.Client()
# Create a colection
l2_norm_collection = chroma_client.get_or_create_collection(
    name="l2_norm",
    metadata={"hnsw:space": "l2"},
)
# Add the vector to the collection
add_embedding(l2_norm_collection, get_all_train_path(POOL_PATH))

#### Search

In [ ]:
def search(query_path, collection, n_result):
  query_image = Image.open(query_path).resize(IMAGE_SIZE)
  query_embedded = create_embedding_one_image(query_image)
  results = collection.query(
      query_embeddings = query_embedded,
      n_results = n_result,
  )
  return results

L1 norm

In [ ]:
query_path = '/content/data/test/African_crocodile/n01697457_18534.JPEG'
search(query_path, l2_norm_collection, 10)